시험 일정 불러오기

In [42]:
import requests
from bs4 import BeautifulSoup

def search_text(text, list):
    #반환할 리스트 생성
    r_list = []
    for str in list: #list안을 반복
        if text in str: #리스트 안에 있는 문자열 중 text 를 포함한 것이 있으면
            r_list.append(str) #반환할 리스트에 추가가
    
    return r_list

def make_test_calender(month):
    #월 입력 받아서 url에 활용
    if month <10:  #url을 만들기 위한 변수 변환환
        month_str = f'0{month}'
    else :
        month_str = f'{month}'
    
    #월간 시험일정 url
    url = f'https://www.q-net.or.kr/crf021.do?id=crf02103&gSite=Q&gId=&schGb=list&schMonth=2025{month_str}01'

    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
    }
    res = requests.get(url, headers=req_header)

    if res.ok: #응답이 있으면
        res.encoding = "utf-8"
        html = res.text
        soup = BeautifulSoup(html, "html.parser")
        #시험일정을 만들기 위한 p태그 검색
        p_tag_list = soup.select("th p.month")
        #날짜 정보만 추려서 리스트 생성
        schedule_list=[]
        for p_tag in p_tag_list:
            schedule_list.append(p_tag.text)
        #시험명을 찾기 위한 a태그 검색 #type1(기술자격), type2(전문자격)
        a_tag_list = soup.select("div[class*=type] a")
        # print(a_tag_list)

        test_list = []
        for a_tag in a_tag_list: #시험명만을 리스트로 만들기 위해 반복문 실행
            temp_list = a_tag['onclick'].split(",")[1:] #앞에 필요없는 거 버리기
            list = temp_list[:-2] #뒤에 날짜 날리기
            list = ", ".join(list)
            test_list.append(list) #test_list에 요소로 추가
        
        test_schedule = dict(zip(schedule_list,test_list)) #시험일정과 시험명을 dictionary로 합침
        
        return test_schedule


    else:
        print(f'Error code: {res.status_code}')


make_test_calender(4)

{'03.31(월)~04.04(금)': "'제34회 공인노무사 1차 원서접수,  제25회 소방시설관리사 1차 원서접수'",
 '04.01(화)~04.02(수)': "'기능장 제77회 실기시험 ,  기능사 제1회 실기시험 '",
 '04.02(수)': "'제23회 청소년상담사 필기 합격자 발표'",
 '04.05(토)': "'제22회 농산물품질관리사 1차 ,  제36회 감정평가사 1차 '",
 '04.05(토)~04.10(목)': "'기능사 제2회 필기시험 '",
 '04.07(월)~04.10(목)': "'기술사 제136회 필기시험 원서접수'",
 '04.07(월)~04.11(금)': "'제11회 수산물품질관리사 1차 원서접수,  제11회 손해평가사 1차 원서접수'",
 '04.09(수)': "'제23회 경매사 1차 합격자 발표'",
 '04.09(수)~04.12(토)': "'제1회 국가유산수리기능자(24종목) 실기 '",
 '04.10(목)~04.11(금)': "'제37회 정수시설운영관리사 1차 빈자리 원서접수,  제37회 정수시설운영관리사 2차 빈자리 원서접수'",
 '04.11(금)': "'기능장 제77회 실기시험 합격예정자 발표,  기능사 제1회 실기시험 합격예정자 발표'",
 '04.12(토)': "'제40회 경영지도사 1차 ,  제40회 기술지도사 1차 '",
 '04.13(일)~04.14(월)': "'기술사 제135회 면접시험 빈자리 원서접수,  기사 제1회 실기시험 빈자리 원서접수'",
 '04.14(월)~04.17(목)': "'기사 제2회 필기시험 원서접수'",
 '04.14(월)~04.18(금)': "'제13회 행정사 1차 원서접수'",
 '04.16(수)': "'기능사 제2회 필기시험 합격예정자 발표'",
 '04.18(금)': "'제23회 가맹거래사 1차 합격자 발표,  제42회 관세사 1차 합격자 발표'",
 '04.19(토)': "'기능장 제77회 실기시험 1차 합격자 발표,  기능사 제1회 실기시험 1차 합격자 발표'",
 '04.19

시험일정 데이터프레임화

In [ ]:
import json
import os

def make_data_folder():
    folder_path = os.path.join(os.getcwd(),'data') # 폴더 경로 만들기
    os.makedirs(folder_path,exist_ok=True) # 만든 경로에 폴더 생성성
month =4
tmp_cal = make_test_calender(month)
def make_json(month):
    with open (f'data/test_schedule_{month}.json','w',encoding='utf-8') as f:
        json.dump(tmp_cal,f)

make_json(month)

In [64]:
import pandas as pd
import json

# Load the JSON file as a dictionary
def read_json(month):
	with open(f'data/test_schedule_{month}.json', 'r', encoding='utf-8') as f:
		return json.load(f)

# Convert the dictionary into a DataFrame
schedule_dict = read_json(4)
schedule_df = pd.DataFrame(list(schedule_dict.items()), columns=['일정', '시험명'])
schedule_df.head()

,일정,시험명
0,03.31(월)~04.04(금),"'제34회 공인노무사 1차 원서접수, 제25회 소방시설관리사 1차 원서접수'"
1,04.01(화)~04.02(수),"'기능장 제77회 실기시험 , 기능사 제1회 실기시험 '"
2,04.02(수),'제23회 청소년상담사 필기 합격자 발표'
3,04.05(토),"'제22회 농산물품질관리사 1차 , 제36회 감정평가사 1차 '"
4,04.05(토)~04.10(목),'기능사 제2회 필기시험 '


출력

In [ ]:
def print_Df(tag):
    test_list = schedule_df['시험명']
    wanted = search_text(tag,test_list)
    schedule_df[schedule_df['시험명'].isin(wanted)].reset_index(drop=True)



,일정,시험명
0,04.13(일)~04.14(월),"'기술사 제135회 면접시험 빈자리 원서접수, 기사 제1회 실기시험 빈자리 원서접수'"
1,04.14(월)~04.17(목),'기사 제2회 필기시험 원서접수'
2,04.21(월)~04.24(목),"'기술사 제135회 면접시험 , 기사 제1회 실기시험 '"
